<a href="https://colab.research.google.com/github/tomasz-soltysiak/NLP_IndianLanguage/blob/master/NLP_Nueral_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import matplotlib.patches as mpatches
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.model_selection import cross_val_score
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score,roc_curve,confusion_matrix,roc_auc_score, mean_squared_error
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re

In [88]:
df_train=pd.read_csv('/content/tamil_movie_reviews_train.csv',delimiter=',')
df_test=pd.read_csv('/content/tamil_movie_reviews_test.csv',delimiter=',')
df_train.shape

(480, 3)

In [89]:
import re
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
#stop_words = set(stopwords.words('')) 
porter=PorterStemmer()
lemmatizer = WordNetLemmatizer()
def clean_text(text):
    #text = re.sub(r'[^\w\s]','',text, re.UNICODE)
    #text = text.lower()
    #text = [porter.stem(word) for word in text]
    #text = [lemmatizer.lemmatize(token) for token in text]
    #text = [lemmatizer.lemmatize(token, "v") for token in text]
    #text = [word for word in text if not word in stop_words]
    text = " ".join(text)
    return text

df_train['ReviewInTamil1'] = df_train.ReviewInTamil.apply(lambda x: clean_text(x))
df_test['ReviewInTamil1'] = df_test.ReviewInTamil.apply(lambda x: clean_text(x))
df_train.head()







,ReviewId,ReviewInTamil,Rating,ReviewInTamil1
0,408,"தமிழ் சினிமாவில் ஒரு சிலர் மட்டுமே பணம், பிஸின...",4.00,த ம ி ழ ் ச ி ன ி ம ா வ ி ல ் ஒ ர ு ச ி ...
1,107,கரு : சினிமா படப்பிடிப்புகளில் வெளிச்சம் பாய்ச...,2.00,க ர ு : ச ி ன ி ம ா ப ட ப ் ப ி ட ி ப ் ...
2,319,தமிழ் சினிமாவில் தற்போது நாயகர்களுக்கு இணையாக ...,3.25,த ம ி ழ ் ச ி ன ி ம ா வ ி ல ் த ற ் ப ோ த ...
3,484,உலக அளவில் அனைத்து தரப்பினரையும் தன் நடிப்பால்...,2.25,உ ல க அ ள வ ி ல ் அ ன ை த ் த ு த ர ப ் ...
4,204,"கரு : வில்லனின் கையாள் , வில்லன் செய்த நம்பிக்...",3.00,க ர ு : வ ி ல ் ல ன ி ன ் க ை ய ா ள ் ...


In [98]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers
max_features = 130

scaler=StandardScaler()
#tokenizer.fit_on_texts(df_train['ReviewInTamil1'])
list_tokenized_train = tokenizer.texts_to_sequences(df_train['ReviewInTamil1'])


#list_tokenized_test = tokenizer.texts_to_sequences(df_test['ReviewInTamil1'])
maxlen = 130

X_train = tokenizer.texts_to_matrix(df_train['ReviewInTamil1'])
#X_train=np.array(scaler.fit_transform(X_train)).reshape(-1,1)
y_train = df_train['Rating']
#y_train=np.array(scaler.fit_transform(y_train))

X_test = tokenizer.texts_to_matrix(df_test['ReviewInTamil1'])
y_test = df_test['Rating']

embed_size = 128
model = Sequential()

model.add(Dense(units=32,input_shape=(130,)))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.05))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse'])
batch_size = 100
epochs = 15
model.fit(X_train,y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2)
'''

model.add(Embedding(max_features, embed_size))
model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(GlobalMaxPool1D())
'''

Train on 384 samples, validate on 96 samples
Epoch 1/15
384/384 [==============================] - 6s 16ms/step - loss: 4.2898 - mean_squared_error: 4.2898 - val_loss: 1.9440 - val_mean_squared_error: 1.9440
Epoch 2/15
384/384 [==============================] - 0s 52us/step - loss: 1.1228 - mean_squared_error: 1.1228 - val_loss: 0.3457 - val_mean_squared_error: 0.3457
Epoch 3/15
384/384 [==============================] - 0s 38us/step - loss: 0.5161 - mean_squared_error: 0.5161 - val_loss: 0.6528 - val_mean_squared_error: 0.6528
Epoch 4/15
384/384 [==============================] - 0s 35us/step - loss: 0.8931 - mean_squared_error: 0.8931 - val_loss: 0.5996 - val_mean_squared_error: 0.5996
Epoch 5/15
384/384 [==============================] - 0s 31us/step - loss: 0.6624 - mean_squared_error: 0.6624 - val_loss: 0.3442 - val_mean_squared_error: 0.3442
Epoch 6/15
384/384 [==============================] - 0s 29us/step - loss: 0.4417 - mean_squared_error: 0.4417 - val_loss: 0.3553 - val_mean

'\n\nmodel.add(Embedding(max_features, embed_size))\nmodel.add(Bidirectional(LSTM(32, return_sequences = True)))\nmodel.add(GlobalMaxPool1D())\n'

In [91]:
y_pred=model.predict(X_test)
mean_squared_error(y_test,y_pred)

4.51504254749368

In [0]:
Reg=LinearRegression()
Reg.fit(X_train,y_train)
y_pred=Reg.predict(X_test)
mean_squared_error(y_test,y_pred)

0.4191248870092975

In [94]:
X_train.shape

(480, 130)